In [2]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In these notes I want to go over some classic Monte Carlo sampling techniques.

### Rejection sampling

This is a really basic sampling technique. Here's how it goes. We want to sample from a target distribution $p(x)$ that might be hard. So we envelope it with another, easier-to-sample distribution $q(x)$, ie

$$ kq(x) \ge p(x) \text{   for all }x$$

In [ ]:
stats.